# Mapping qubits
In this notebook we will cover the QGym `InitialMapping` environment.

This environment is aimed at solving the problem of mapping virtual to physical qubits that have a certain topology.

In [ ]:
%matplotlib inline
import numpy as np
import networkx as nx
from networkx.generators import fast_gnp_random_graph
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from IPython.display import clear_output

from qgym.envs.initial_mapping import InitialMapping
from qgym.envs.initial_mapping.initial_mapping_rewarders import *

In [ ]:
def render_rgb(step, rgb_array):
    clear_output(wait=True)
    plt.figure(figsize=(40, 20))
    plt.title(f"Step {step}", fontsize=40)
    plt.imshow(rgb_array)
    plt.axis("off")
    plt.show()

In [ ]:
connection_graph = nx.Graph()
connection_graph.add_edge(1, 2)
connection_graph.add_edge(1, 3)
connection_graph.add_edge(1, 4)
connection_graph.number_of_nodes()
nx.draw(connection_graph)

In [ ]:
env = InitialMapping(0.5, connection_graph=connection_graph)
env.rewarder = EpisodeRewarder(illegal_action_penalty=0)
check_env(env, warn=True)

model = PPO("MultiInputPolicy", env, verbose=1)

model.learn(int(1e5))

In [ ]:
obs = env.reset(interaction_graph=connection_graph)
for i in range(1000):
    action, states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    render_rgb(i, env.render(mode="rgb_array"))
    if done:
        break

In [ ]:
interaction_graph = connection_graph.copy()
interaction_graph.remove_edge(1, 3)
nx.draw(interaction_graph)

In [ ]:
obs = env.reset(interaction_graph=interaction_graph)
for i in range(1000):
    action, states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    render_rgb(i, env.render(mode="rgb_array"))
    if done:
        break

In [ ]:
interaction_graph = connection_graph.copy()
interaction_graph.add_edge(3, 2)
nx.draw(interaction_graph)

In [ ]:
obs = env.reset(interaction_graph=interaction_graph)
for i in range(1000):
    action, states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    render_rgb(i, env.render(mode="rgb_array"))
    if done:
        break

In [ ]:
connection_graph = nx.Graph()
connection_graph.add_edge(0, 1)
connection_graph.add_edge(1, 2)
connection_graph.add_edge(2, 0)
connection_graph.add_edge(2, 3)
connection_graph.add_edge(3, 4)
connection_graph.add_edge(4, 2)
connection_graph.number_of_nodes()
nx.draw(connection_graph)

In [ ]:
env = InitialMapping(0.5, connection_graph=connection_graph)
env.rewarder = EpisodeRewarder(illegal_action_penalty=-10)
check_env(env, warn=True)

model = PPO("MultiInputPolicy", env, verbose=1)

model.learn(int(1e6))
model.save("saved_model")

In [ ]:
interaction_graph = fast_gnp_random_graph(5, 0.5)
nx.draw(interaction_graph)

In [ ]:
env = InitialMapping(0.5, connection_graph=connection_graph)
model = PPO.load("saved_model")
obs = env.reset(interaction_graph=interaction_graph)
for i in range(1000):
    action, states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    render_rgb(i, env.render(mode="rgb_array"))
    if done:
        break

In [ ]:
# try the same with the other two rewarders that we have defined

In [ ]:
# QAP rewarder